In [0]:
# ! pip install -q kaggle

In [0]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [0]:
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/

In [0]:
# ! chmod 600 ~/.kaggle/kaggle.json

In [0]:
# !kaggle datasets download -d sudalairajkumar/novel-corona-virus-2019-dataset

In [0]:
# !unzip novel-corona-virus-2019-dataset.zip

In [1]:
!wget https://raw.githubusercontent.com/MeMihir/nCoViD-Viz/master/covid_19_data/covid_19_data.csv

--2020-05-27 05:19:34--  https://raw.githubusercontent.com/MeMihir/nCoViD-Viz/master/covid_19_data/covid_19_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1947188 (1.9M) [text/plain]
Saving to: ‘covid_19_data.csv’

covid_19_data.csv   100%[===================>]   1.86M  8.83MB/s    in 0.2s    

2020-05-27 05:19:34 (8.83 MB/s) - ‘covid_19_data.csv’ saved [1947188/1947188]



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import cufflinks as cf

# For Notebooks
cf.go_offline()

In [0]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [0]:
df = pd.read_csv('./covid_19_data.csv')
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Recovered']
df['ObservationDate'] = pd.to_datetime(df['ObservationDate'])

In [25]:
df

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Active
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,1.0
1,2,2020-01-22,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0,14.0
2,3,2020-01-22,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0,6.0
3,4,2020-01-22,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,1.0
4,5,2020-01-22,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
29421,29422,2020-05-25,Yucatan,Mexico,2020-05-26 02:32:35,1522.0,123.0,1009.0,390.0
29422,29423,2020-05-25,Yukon,Canada,2020-05-26 02:32:35,11.0,0.0,11.0,0.0
29423,29424,2020-05-25,Yunnan,Mainland China,2020-05-26 02:32:35,185.0,2.0,183.0,0.0
29424,29425,2020-05-25,Zacatecas,Mexico,2020-05-26 02:32:35,243.0,28.0,161.0,54.0


In [0]:
def non_cumulative(l):
    for i in range (len(l)-1, 0, -1):
        l[i] -= l[i-1]
    return l

In [0]:
def sort_by_country(country):
    temp_df = df[df['Country/Region'] == country]
    temp_df = temp_df.drop('Province/State', axis=1).drop('SNo', axis=1)
    temp_df = temp_df.groupby(['Country/Region', 'ObservationDate'], as_index=False).aggregate(['sum'], )
    temp_df.columns = df.columns[4:]
    temp_df= temp_df[temp_df['Confirmed'] != 0]
    temp_df['ConfirmedPerDay'] = non_cumulative(temp_df['Confirmed'].copy())
    temp_df['DeathsPerDay'] = non_cumulative(temp_df['Deaths'].copy())
    temp_df['RecoveredPerDay'] = non_cumulative(temp_df['Recovered'].copy())
    temp_df['Country'] = [country]*temp_df.shape[0]
    return temp_df

In [0]:
df_country = sort_by_country(df['Country/Region'].value_counts().index[0])

for i in range(1, len(df['Country/Region'].value_counts())):
    temp = sort_by_country(df['Country/Region'].value_counts().index[i])
    df_country = pd.concat([df_country, temp])

df_country['Date'] = list(map(lambda x: x[1] ,df_country.index.values))
df_country = df_country.sort_index()
# df_country

In [0]:
df['ConfirmedPerDay'] = non_cumulative(df['Confirmed'].copy())
df['DeathsPerDay'] = non_cumulative(df['Deaths'].copy())
df['RecoveredPerDay'] = non_cumulative(df['Recovered'].copy())
# df

In [0]:
dates = pd.Series(df_country['Date'].unique())
total_cases = df_country[['Confirmed','ConfirmedPerDay', 'Deaths','DeathsPerDay', 'Recovered','RecoveredPerDay', 'Active', 'Date']]
total_cases = total_cases.groupby('Date').sum()

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28974 entries, 0 to 28973
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   SNo              28974 non-null  int64         
 1   ObservationDate  28974 non-null  datetime64[ns]
 2   Province/State   14253 non-null  object        
 3   Country/Region   28974 non-null  object        
 4   Last Update      28974 non-null  object        
 5   Confirmed        28974 non-null  float64       
 6   Deaths           28974 non-null  float64       
 7   Recovered        28974 non-null  float64       
 8   Active           28974 non-null  float64       
 9   ConfirmedPerDay  28974 non-null  float64       
 10  DeathsPerDay     28974 non-null  float64       
 11  RecoveredPerDay  28974 non-null  float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(3)
memory usage: 2.7+ MB


In [31]:
map_df = df_country.copy()
map_df['DateStr'] = map_df['Date'].apply(lambda x: str(x).split(' ')[0])
map_df.sort_values(by='Date', inplace=True)
map_df

,,Last Update,Confirmed,Deaths,Recovered,Active,ConfirmedPerDay,DeathsPerDay,RecoveredPerDay,Country,Date,DateStr
Country/Region,ObservationDate,,,,,,,,,,,
Japan,2020-01-22,1/22/2020 17:00,2.0,0.0,0.0,2.0,2.0,0.0,0.0,Japan,2020-01-22,2020-01-22
Macau,2020-01-22,1/22/2020 17:00,1.0,0.0,0.0,1.0,1.0,0.0,0.0,Macau,2020-01-22,2020-01-22
US,2020-01-22,1/22/2020 17:00,1.0,0.0,0.0,1.0,1.0,0.0,0.0,US,2020-01-22,2020-01-22
South Korea,2020-01-22,1/22/2020 17:00,1.0,0.0,0.0,1.0,1.0,0.0,0.0,South Korea,2020-01-22,2020-01-22
Mainland China,2020-01-22,1/22/2020 17:001/22/2020 17:001/22/2020 17:001...,547.0,17.0,28.0,502.0,547.0,17.0,28.0,Mainland China,2020-01-22,2020-01-22
...,...,...,...,...,...,...,...,...,...,...,...,...
Slovakia,2020-05-25,2020-05-26 02:32:35,1511.0,28.0,1307.0,176.0,2.0,0.0,6.0,Slovakia,2020-05-25,2020-05-25
Romania,2020-05-25,2020-05-26 02:32:35,18283.0,1205.0,11630.0,5448.0,213.0,20.0,231.0,Romania,2020-05-25,2020-05-25
Timor-Leste,2020-05-25,2020-05-26 02:32:35,24.0,0.0,24.0,0.0,0.0,0.0,0.0,Timor-Leste,2020-05-25,2020-05-25


In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
px.choropleth(map_df, locations="Country", 
                    locationmode='country names', color="Confirmed", 
                    hover_name="Country", 
                    animation_frame='DateStr',
                    # color_continuous_scale="peach", 
                    title='Countries with Confirmed Cases')

In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
px.choropleth(map_df, locations="Country", 
                    locationmode='country names', color="Deaths", 
                    hover_name="Country",
                    animation_frame='DateStr',
                    # color_continuous_scale="peach", 
                    title='Countries with Fatalities')

In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
px.choropleth(map_df, locations="Country", 
                    locationmode='country names', color="Recovered", 
                    hover_name="Country",
                    animation_frame='DateStr',
                    # color_continuous_scale="peach", 
                    title='Countries with Confirmed Cases')

In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
px.choropleth(map_df, locations="Country", 
                    locationmode='country names', color="Active", 
                    hover_name="Country",
                    animation_frame='DateStr',
                    # color_continuous_scale="peach", 
                    title='Countries with Confirmed Cases')

In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
fig = px.bar(df_country, x='Date', y='Confirmed', color='Country')
fig.show()

In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
fig = px.bar(df_country, x='Date', y='ConfirmedPerDay', color='Country')
fig.show()

In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
fig = px.bar(df_country, x='Date', y='Deaths', color='Country')
fig.show()

In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
fig = px.bar(df_country, x='Date', y='DeathsPerDay', color='Country')
fig.show()

In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
fig = px.bar(df_country, x='Date', y='Recovered', color='Country')
fig.show()

In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
fig = px.bar(df_country, x='Date', y='RecoveredPerDay', color='Country')
fig.show()

In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
fig = px.bar(df_country, x='Date', y='Active', color='Country')
fig.show()

In [35]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
px.line(df_country, x='Confirmed', y='ConfirmedPerDay', color='Country', log_x=True, log_y=True)

In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
country = input('Enter Country \t: ')
spread_data = df_country[df_country['Country']==country]
spread_data.set_index('Date', inplace=True)
spread_data[['Confirmed', 'Deaths', 'Recovered']].iplot(kind='spread')

In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
spread_data[['ConfirmedPerDay', 'DeathsPerDay', 'RecoveredPerDay']].iplot(kind='spread')

In [5]:
from pandas_datareader import data

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [0]:
start_date = '-01-15'
end_date = '-05-24'
companies = ['TSLA', 'MSFT', 'AMZN', 'AAPL', 'NFLX', 'BABA']

In [0]:
# stocks20 = data.DataReader(companies, 'yahoo', f'2020{start_date}', f'2020{end_date}')
# stocks19 = data.DataReader(companies, 'yahoo', f'2019{start_date}', f'2019{end_date}')
# stocks18 = data.DataReader(companies, 'yahoo', f'2018{start_date}', f'2018{end_date}')

def stockCompare(company, attr='Close'):
  stocks20 = data.DataReader(company, 'yahoo', f'2020{start_date}', f'2020{end_date}')
  stocks19 = data.DataReader(company, 'yahoo', f'2019{start_date}', f'2019{end_date}')
  stocks18 = data.DataReader(company, 'yahoo', f'2018{start_date}', f'2018{end_date}')

  print(stocks20.shape, stocks19.shape, stocks18.shape)
  stocks18 = pd.DataFrame(stocks18[attr])
  stocks18.columns = ['2018']
  stocks18['DDMM'] = pd.Series(stocks18.index.values).apply(lambda x: '2020-'+str(x).split(' ')[0][5:]).values

  stocks19 = pd.DataFrame(stocks19[attr])
  stocks19.columns = ['2019']
  stocks19['DDMM'] = pd.Series(stocks19.index.values).apply(lambda x: '2020-'+str(x).split(' ')[0][5:]).values

  stocks20 = pd.DataFrame(stocks20[attr])
  stocks20.columns = ['2020']
  stocks20['DDMM'] = pd.Series(stocks20.index.values).apply(lambda x: '2020-'+str(x).split(' ')[0][5:]).values

  compare = pd.merge(left=stocks18, right=stocks19, on='DDMM', how='outer')
  compare = pd.merge(left=compare, right=stocks20, on='DDMM', how='outer')
  compare.set_index('DDMM', inplace=True)
  compare.fillna(method='ffill', inplace=True)
  compare.sort_index(inplace=True)
  return compare

In [0]:
nflx = stockCompare('NFLX') #straming service

(90, 6) (91, 6) (91, 6)


In [0]:
amazon = stockCompare('AMZN') #ecom websites / streaming service

(86, 6) (87, 6) (87, 6)


In [0]:
sensex = stockCompare('^BSESN') #sensex

(86, 6) (86, 6) (89, 6)


In [0]:
nifty50 = stockCompare('^NSEI') #nifty

(81, 6) (81, 6) (84, 6)


In [0]:
lufthansa = stockCompare('LHA.DE') #airline

(85, 6) (86, 6) (86, 6)


In [0]:
indigo = stockCompare('INDIGO.NS') #airline

(81, 6) (81, 6) (84, 6)


In [0]:
# jetAirways = stockCompare('JETAIRWAYS.NS') #airline

(81, 6) (81, 6) (84, 6)


In [0]:
walmart = stockCompare('WMT') #retail

(86, 6) (87, 6) (87, 6)


In [0]:
dmart = stockCompare('DMART.NS') #retail

(81, 6) (81, 6) (84, 6)


In [0]:
reliance = stockCompare('RELIANCE.NS') #retail

(81, 6) (81, 6) (84, 6)


In [0]:
tataMotors = stockCompare('TTM') #tata motors

(86, 6) (87, 6) (87, 6)


In [0]:
tesla = stockCompare('TSLA') #automobile

(86, 6) (87, 6) (87, 6)


In [0]:
volkswagon = stockCompare('VOW3.DE') #automobile

(85, 6) (86, 6) (86, 6)


In [0]:
ford = stockCompare('F') #automobile

(90, 6) (91, 6) (91, 6)


In [0]:
dlf = stockCompare('DLF.NS') #Real Estate

(86, 6) (86, 6) (89, 6)


In [0]:
oberoi = stockCompare('OBEROIRLTY.NS') #Real Estate

(86, 6) (86, 6) (89, 6)


In [0]:
abcapital = stockCompare('ABCAPITAL.NS') #Finance

(86, 6) (86, 6) (89, 6)


In [0]:
muthoot = stockCompare('MUTHOOTFIN.NS') #Finance

(86, 6) (86, 6) (89, 6)


In [0]:
bajaj = stockCompare('BAJFINANCE.NS') #Finance

(86, 6) (86, 6) (89, 6)


In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
sensex[['2020', '2018', '2019']].iplot(kind='spread')

/usr/local/lib/python3.6/dist-packages/cufflinks/plotlytools.py:849: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead

/usr/local/lib/python3.6/dist-packages/cufflinks/plotlytools.py:850: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



In [0]:
stocks_affect = total_cases.join(sensex)
stocks_affect['ConfirmedPerDay'] = stocks_affect['ConfirmedPerDay']/max(stocks_affect['ConfirmedPerDay'])
stocks_affect['2020'] = stocks_affect['2020']/max(stocks_affect['2020'])

In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
stocks_affect[['ConfirmedPerDay', '2020']].iplot(kind='spread')

/usr/local/lib/python3.6/dist-packages/cufflinks/plotlytools.py:849: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead

/usr/local/lib/python3.6/dist-packages/cufflinks/plotlytools.py:850: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



In [0]:
india = df_country[df_country['Country'] == 'India']
india = total_cases.join(sensex)
india['ConfirmedPerDay'] = india['ConfirmedPerDay']/max(india['ConfirmedPerDay'])
india['2020'] = india['2020']/max(india['2020'])

In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
stocks_affect[['ConfirmedPerDay', '2020']].iplot(kind='spread')

/usr/local/lib/python3.6/dist-packages/cufflinks/plotlytools.py:849: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead

/usr/local/lib/python3.6/dist-packages/cufflinks/plotlytools.py:850: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



In [0]:
curr_start_date = '-01-01'
curr_end_date = '-05-24'

In [0]:
currUSD = data.DataReader(['INR=X', 'EUR=X', 'JPY=X', 'KRW=X', 'CNY=X'], 'yahoo', f'2020{curr_start_date}', f'2020{curr_end_date}')

In [16]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
currUSD['Close'].iplot(kind='spread')

/usr/local/lib/python3.6/dist-packages/cufflinks/plotlytools.py:850: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



In [0]:
currINR = data.DataReader(['INRCNY=X', 'INREUR=X', 'INRJPY=X', 'INRKRW=X', 'INRUSD=X'], 'yahoo', f'2020{curr_start_date}', f'2020{curr_end_date}')

In [37]:
configure_plotly_browser_state()
init_notebook_mode(connected=True)
currINR['Close'].iplot(kind='spread')

/usr/local/lib/python3.6/dist-packages/cufflinks/plotlytools.py:850: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead

